<a href="https://colab.research.google.com/github/philorfa/Biodata/blob/main/BelgiumClassificationcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#@title
from __future__ import absolute_import, division, print_function, unicode_literals # legacy compatibility

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
#@title
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import backend 
from keras.utils import np_utils

from numpy import savetxt


Requirement already up-to-date: pyswarm in /usr/local/lib/python3.7/dist-packages (0.6)
Requirement already up-to-date: pactools in /usr/local/lib/python3.7/dist-packages (0.3.1)
Running on PyMC3 v3.11.2


In [29]:
X= pd.read_csv('/content/drive/MyDrive/Biodata/Datas/Belgium/X_train_Bel.csv')
X=X.drop(['Unnamed: 0'],axis=1)
y = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/Belgium/y_train_Bel.csv')
y=y.drop(['Unnamed: 0'],axis=1)

Test_data = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/Belgium/X_test_Bel.csv')
Test_data = Test_data.drop(['Unnamed: 0'],axis=1)

Test_labels = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/Belgium/y_test_Bel.csv')
Test_labels = Test_labels.drop(['Unnamed: 0'],axis=1)


In [30]:
print("Shape X_train:", X.shape)
print("Shape y_train:", y.shape)

print("Shape X_test:", Test_data.shape)
print("Shape y_test:", Test_labels.shape)

Shape X_train: (16469, 64)
Shape y_train: (16469,)
Shape X_test: (4118, 64)
Shape y_test: (4118, 1)


In [18]:
for i in range(len(y)):
  if (y['Depression_Score'][i]<=7.0):
    y['Depression_Score'][i]=0
  elif (y['Depression_Score'][i]<=12.0):
    y['Depression_Score'][i]=1
  else:
    y['Depression_Score'][i]=2


for i in range(len(Test_labels)):
  if (Test_labels['Depression_Score'][i]<=7.0):
    Test_labels['Depression_Score'][i]=0
  elif (Test_labels['Depression_Score'][i]<=12.0):
    Test_labels['Depression_Score'][i]=1
  else:
    Test_labels['Depression_Score'][i]=2

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y['Depression_Score'], test_size=0.2, random_state=42)

In [20]:
'''
X_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_train_Wor.csv')
y_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_train_Wor.csv')

X_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_test_Wor.csv')
y_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_test_Wor.csv')
'''

"\nX_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_train_Wor.csv')\ny_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_train_Wor.csv')\n\nX_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_test_Wor.csv')\ny_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_test_Wor.csv')\n"

In [21]:
img_rows, img_cols = 8,8

X_train_cnn=np.array(X_train)
X_test_cnn=np.array(X_test)

X_train_cnn = X_train_cnn.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test_cnn = X_test_cnn.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train_cnn = X_train_cnn.astype('float32')
X_test_cnn = X_test_cnn.astype('float32')

print('x_train shape:', X_train_cnn.shape)
print(X_train_cnn.shape[0], 'train samples')
print(X_test_cnn.shape[0], 'test samples')

x_train shape: (13175, 8, 8, 1)
13175 train samples
3294 test samples


In [22]:
EarlyStopper = EarlyStopping(patience=20, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6,patience=20, min_lr=0.000001)
count=0
max=0
lb=[0,0,0,0,0,0]
ub=[0.2,0.2,0.2,0.2,3,3]

In [23]:
def create_model(x):
  print(x[0],x[1],x[2],x[3],x[4],x[5])
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape = input_shape))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(x[0]))
  if (x[4]<=1):
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(x[2]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
  elif (x[4]<=2):
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(x[2]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
  else:
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(0.2))


  model.add(keras.layers.Dense(3,activation="softmax"))
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
  
  return model

In [26]:
def apple(x):
  global max
  model = create_model(x)
  if (x[5]<1):
    model.fit(X_train_cnn, y_train, epochs=10, batch_size=3000, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  elif (x[5]<2):
    model.fit(X_train_cnn, y_train, epochs=10, batch_size=1000, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  else:
    model.fit(X_train_cnn, y_train, epochs=10, batch_size=500, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  if acc>0.59:
    global count 
    count = count+1
    model.save(f"/content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-{count}-{round(acc, 3)}-{round(loss, 3)}")
    savetxt(f"/content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/data_cnn-{count}.csv", x, delimiter=',')
  return 1/(1+acc)

In [27]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)

0.021907005642980006 0.13144648732490563 0.025417763166880093 0.1254072220326556 2.9690362348223855 2.1018263231588175
Epoch 1/10
27/27 [==============================] - 4s 121ms/step - loss: 1.1005 - accuracy: 0.3721 - val_loss: 1.0047 - val_accuracy: 0.4927
Epoch 2/10
27/27 [==============================] - 3s 104ms/step - loss: 1.0201 - accuracy: 0.4650 - val_loss: 0.9723 - val_accuracy: 0.4879
Epoch 3/10
27/27 [==============================] - 3s 103ms/step - loss: 1.0066 - accuracy: 0.4643 - val_loss: 0.9598 - val_accuracy: 0.4964
Epoch 4/10
27/27 [==============================] - 3s 103ms/step - loss: 0.9887 - accuracy: 0.4736 - val_loss: 0.9595 - val_accuracy: 0.4985
Epoch 5/10
27/27 [==============================] - 3s 103ms/step - loss: 0.9890 - accuracy: 0.4688 - val_loss: 0.9530 - val_accuracy: 0.4988
Epoch 6/10
27/27 [==============================] - 3s 102ms/step - loss: 0.9753 - accuracy: 0.4747 - val_loss: 0.9817 - val_accuracy: 0.4936
Epoch 7/10
27/27 [===========

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-1-0.592-0.843/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-1-0.592-0.843/assets


0.0664072343534295 0.14984081963633109 0.13339957298735758 0.19913228912723568 1.4725995323629593 1.0093347489837234
Epoch 1/10
14/14 [==============================] - 4s 224ms/step - loss: 1.1273 - accuracy: 0.3556 - val_loss: 1.0807 - val_accuracy: 0.3543
Epoch 2/10
14/14 [==============================] - 3s 191ms/step - loss: 1.0634 - accuracy: 0.3918 - val_loss: 0.9416 - val_accuracy: 0.5143
Epoch 3/10
14/14 [==============================] - 3s 194ms/step - loss: 0.9514 - accuracy: 0.5005 - val_loss: 0.8702 - val_accuracy: 0.5832
Epoch 4/10
14/14 [==============================] - 3s 189ms/step - loss: 0.9219 - accuracy: 0.5340 - val_loss: 0.8602 - val_accuracy: 0.5801
Epoch 5/10
14/14 [==============================] - 3s 188ms/step - loss: 0.9048 - accuracy: 0.5521 - val_loss: 0.8469 - val_accuracy: 0.5920
Epoch 6/10
14/14 [==============================] - 3s 184ms/step - loss: 0.8762 - accuracy: 0.5688 - val_loss: 0.8476 - val_accuracy: 0.5920
Epoch 7/10
14/14 [=============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-2-0.591-0.82/assets


...

0.017333626000375846 0.06362584486094212 0.19281106664134343 0.08393000915037672 2.8080805242737283 2.489247200769238
Epoch 1/10
27/27 [==============================] - 4s 120ms/step - loss: 1.1581 - accuracy: 0.3453 - val_loss: 1.0980 - val_accuracy: 0.3543
Epoch 2/10
27/27 [==============================] - 3s 104ms/step - loss: 1.0979 - accuracy: 0.3520 - val_loss: 1.0973 - val_accuracy: 0.3543
Epoch 3/10
27/27 [==============================] - 3s 103ms/step - loss: 1.0970 - accuracy: 0.3526 - val_loss: 1.0967 - val_accuracy: 0.3543
Epoch 4/10
27/27 [==============================] - 3s 101ms/step - loss: 1.0965 - accuracy: 0.3490 - val_loss: 1.0963 - val_accuracy: 0.3543
Epoch 5/10
27/27 [==============================] - 3s 103ms/step - loss: 1.0956 - accuracy: 0.3483 - val_loss: 1.0961 - val_accuracy: 0.3467
Epoch 6/10
27/27 [==============================] - 3s 102ms/step - loss: 1.0960 - accuracy: 0.3522 - val_loss: 1.0960 - val_accuracy: 0.3467
Epoch 7/10
27/27 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-5-0.591-0.822/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/Belgium/Models_class/model_cnn-5-0.591-0.822/assets


0.1978452596537437 0.03664306123984158 0.008380252610973393 0.18775634494905258 2.6737401009626955 0.30065677319050665
Epoch 1/10
5/5 [==============================] - 5s 585ms/step - loss: 1.1082 - accuracy: 0.3392 - val_loss: 1.0686 - val_accuracy: 0.3543
Epoch 2/10
5/5 [==============================] - 2s 483ms/step - loss: 1.0931 - accuracy: 0.3699 - val_loss: 1.0979 - val_accuracy: 0.3777
Epoch 3/10
5/5 [==============================] - 2s 474ms/step - loss: 1.0851 - accuracy: 0.3808 - val_loss: 1.0212 - val_accuracy: 0.4590
Epoch 4/10
5/5 [==============================] - 2s 477ms/step - loss: 1.0370 - accuracy: 0.4317 - val_loss: 0.9770 - val_accuracy: 0.4800
Epoch 5/10
5/5 [==============================] - 2s 479ms/step - loss: 0.9946 - accuracy: 0.4571 - val_loss: 0.9544 - val_accuracy: 0.4915
Epoch 6/10
5/5 [==============================] - 2s 477ms/step - loss: 0.9844 - accuracy: 0.4575 - val_loss: 0.9451 - val_accuracy: 0.4954
Epoch 7/10
5/5 [=========================

KeyboardInterrupt: ignored